# Testing Number of Passes Calculation

In [1]:
%load_ext autoreload
%autoreload 2

In [49]:
import geopandas as gpd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
import pickle
import folium
import osmnx as ox
import folium.plugins
from params import PLOW_SPEED_HIGHWAY, PLOW_SPEED_RESIDENTIAL


In [5]:
def get_salt_from_length(length: float) -> float:
    """
    Returns the amount of salt (in lbs) required for a given road length in meters. 250 lbs/mile

    Parameters:
        length (float): The length of the road segment.

    Returns:
        float: The amount of salt required for the road segment.
    """
    length_mile = length * 0.000621371
    return length_mile * 250

def is_culdesac(G: nx.MultiDiGraph, node: int) -> bool:
    """
    Determines if a given node in a MultiDiGraph represents a cul-de-sac.
    Parameters:
        G (nx.MultiDiGraph): The MultiDiGraph representing the road network.
        node (int): The node to check.
    Returns:
        bool: True if the node is a cul-de-sac, False otherwise.
    """

    if G.out_degree(node) == 0:
        return True
    
    edge = list(G.edges(node, data=True))
    attrb = edge[0][2]

    # if 'roadtype' in attrb:
    #     if attrb['roadtype'] == 'Ct' or attrb['roadtype'] == 'Cir':
    #         if attrb['highway'] == 'residential':
    #             return True

    return G.out_degree(node) == 1 and G.in_degree(node) == 1 and attrb['highway'] == 'residential' and attrb['reversed'] == True and attrb['length'] < 500

In [52]:
street_gdf = gpd.read_file("C:\\Users\\Sneez\\Desktop\\Snowplowing\\Snowplow-Routing-Middleton\\Snowplow-Routing-Middleton\\graph_data\\OSMWithData.gpkg")

# Read the OSM Graph
G = pickle.load(open("C:\\Users\\Sneez\\Desktop\\Snowplowing\\Snowplow-Routing-Middleton\\Snowplow-Routing-Middleton\\graph_data\\streets_graph.pickle", 'rb'))
G = nx.convert_node_labels_to_integers(G)

nodes, edges = ox.graph_to_gdfs(G) # better than momepy b/c fills in missing geometry attributes

edges['jurisdiction'] = np.array(street_gdf['Jurisdicti'])
edges['width'] = np.array(street_gdf['With_EE_ft'])
edges['roadtype'] = np.array(street_gdf['abvPostTyp'])
edges['maintainer'] = np.array(street_gdf['Maintained'])
G = ox.graph_from_gdfs(nodes, edges)
priority_keys = {"motorway_link":1, "tertiary_link":1, "secondary_link":1, "primary_link":1, "unclassified":1, "residential":2, "tertiary":3, "secondary":4, "primary":5, "motorway":6}
passes_keys = {"motorway_link":1, "tertiary_link":1, "secondary_link":1, "primary_link":1, "unclassified":1, "residential":2, "tertiary":2, "secondary":3, "primary":3, "motorway":6}

In [54]:
passes_keys = {"motorway_link":1, "tertiary_link":1, "secondary_link":1, "primary_link":1, "unclassified":1, "residential":2, "tertiary":2, "secondary":3, "primary":3, "motorway":6}
small_roads = ["motorway_link", "tertiary_link", "secondary_link", "primary_link", "unclassified"]
def calc_passes(oneway: bool, width: float, highway: str, roadtype: str) -> int:
    """
    Calculates the number of passes required for a given road segment.

    Parameters:
        oneway (bool): Whether the road segment is one-way.
        width (float): The width of the road segment.
        highway (str): The type of the road segment.

    Returns:
        int: The number of passes required for the road segment.
    """
    if highway in small_roads:
        return 1
    if np.isnan(width) and roadtype != "Blvd":
        return passes_keys[highway]
    
    if roadtype == "Blvd":
        return 2 if oneway else 4

    if width <= 36:
        return 1 if oneway else 2
    else:
        return 1 if oneway else 3

In [55]:

priorities = np.empty(len(edges))
passes = np.empty(len(edges))
salt = np.empty(len(edges))
serviced = np.empty(len(edges), dtype=bool)
culdesac = np.empty(len(edges), dtype=bool)
plow_time = np.empty(len(edges))
# go through each edge and update dictionary
for index, edges_data in enumerate(edges.iterrows()):
    edge = edges_data[0]
    data = edges_data[1]

    highway_type = data['highway']
    length_meters = data['length']
    width = data['width']
    roadtype = data['roadtype']
    oneway = data['oneway']
    
    if street_gdf.iloc[index]['Jurisdicti'] == "City":
        priorities[index] = priority_keys[highway_type]
        passes[index] = calc_passes(oneway, width, highway_type, roadtype)
        salt[index] = get_salt_from_length(length_meters)
        serviced[index] = False
    else:
        priorities[index] = 0
        passes[index] = 0
        salt[index] = 0
        serviced[index] = True
    
    if highway_type == "residential":
        plow_time[index] = length_meters / PLOW_SPEED_RESIDENTIAL
    else:
        plow_time[index] = length_meters / PLOW_SPEED_HIGHWAY
    culdesac[index] = is_culdesac(G, edge[1]) # edge[1] corresponds to the second node of the edge
edges['priority'] = priorities
edges['passes_rem'] = passes
edges['salt_per'] = salt
edges['serviced'] = serviced
edges['culdesac'] = culdesac
edges['travel_time'] = plow_time

In [56]:
G = ox.graph_from_gdfs(nodes, edges)
for edge in G.edges(data=True, keys=True):
    if 'roadtype' in edge[3]:
        if (edge[3]['roadtype'] == "Ct" or edge[3]['roadtype'] == "Cir") and edge[3]['highway'] == 'residential':
            edge[3]['culdesac'] = True
    elif "name" in edge[3]:
        if edge[3]['name'] == "Bunker Hill Lane" or edge[3]['name'] == "Patrick Henry Way":
            edge[3]['culdesac'] = True
scc = list(nx.strongly_connected_components(G)) # strongly connected components
scc.remove(max(scc, key=len))

for i in scc:
    for j in i:
        G.remove_node(j) # remove all but the strongest connected component from G

LINESTRING (-89.5181058 43.1115105, -89.5183276 43.1115019, -89.5183992 43.1115009)
LINESTRING (-89.5210677 43.1114563, -89.521805 43.111448, -89.5253697 43.1114083, -89.5266183 43.1113955, -89.5289523 43.1113716, -89.5291454 43.111369)
LINESTRING (-89.5211093 43.1113382, -89.5209208 43.1113376, -89.5204818 43.1113424, -89.5201156 43.1113464, -89.5193477 43.1113491, -89.5192224 43.1113479, -89.5188922 43.1113448, -89.5184086 43.1113461, -89.5183249 43.1113479, -89.5180866 43.1113502)
LINESTRING (-89.5291454 43.111369, -89.5292921 43.1113681)
LINESTRING (-89.5378871 43.1111513, -89.5376462 43.1111513)
LINESTRING (-89.5291444 43.1112539, -89.52894 43.1112572, -89.5276708 43.1112685, -89.5265467 43.1112816, -89.5228376 43.1113149, -89.5220611 43.1113236, -89.521792 43.1113266, -89.5211093 43.1113382)
LINESTRING (-89.5379845 43.1111536, -89.5378871 43.1111513)
LINESTRING (-89.5183992 43.1115009, -89.519835 43.1114754, -89.520924 43.1114586, -89.5210677 43.1114563)
LINESTRING (-89.5292907 4

In [57]:
m = folium.Map(location=[43.1, -89.5], zoom_start=12)
count = 0
for lstring in edges[edges['name'] == 'Airport Road']['geometry']:
    lstring = lstring.__class__([(y, x) for x, y in lstring.coords])
    midpoint = len(list(lstring.coords))//2
    icon_number = folium.plugins.BeautifyIcon(
        border_color="blue",
        border_width=1,
        text_color="blue",
        number=count,
        inner_icon_style="margin-top:2;",
    )
    folium.PolyLine(locations=lstring.coords, weight=1,tooltip=count).add_to(m)
    folium.Marker(location=lstring.coords[midpoint], popup=f"Edge {count}", icon=icon_number).add_to(m)
    count += 1

m

Takeaway: oneway == "True" means that we halve the number of passes. Oneway == "false" means we keep it normal.